In [50]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from splinter.exceptions import ElementDoesNotExist
import re
import requests
import time
from decimal import Decimal

In [6]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

## Scrape Craigslist Freestuff

In [3]:
# Go to sacramento cl and navigate to 1st item

url = "https://sacramento.craigslist.org/d/free-stuff/search/zip"
browser.visit(url)
browser.click_link_by_partial_href('https://sacramento.craigslist.org')
time.sleep(2)

In [6]:
# Using Splinter to scrape cl and store data in dataframe stuff (clean up condition)

stuff = pd.DataFrame(columns=['title','age', 'location'])

for x in range(1, 4):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find(id="titletextonly").text
    age = soup.find('time', class_="date timeago")["datetime"]
    loc = soup.find(id='map')
    coord = [loc["data-latitude"], loc["data-longitude"]]
    # condit = soup.find('p', class_="attrgroup")
    # cond = ''
    # if (condit and condit.b.text): cond = condit.b.text
    
    stuff.loc[x] = [title, age, coord]
    time.sleep(2)
    browser.click_link_by_partial_text('next')

print(stuff)

                 title                       age                  location
1  Free business books  2019-10-25T22:02:24-0700  [38.534700, -121.445100]
2            2 Pallets  2019-10-20T14:50:57-0700  [38.572461, -121.340218]
3    Free baby formula  2019-10-25T21:13:01-0700  [38.689500, -121.447900]


In [ ]:
stuff["title"]

In [7]:
stuff2 = {}
stuff2["title"] = []
stuff2["age"] = []
stuff2["coord"] = []

for x in range(1, 101):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find(id="titletextonly").text
    age = soup.find('time', class_="date timeago")["datetime"]
    loc = soup.find(id='map')
    coord = [loc["data-latitude"], loc["data-longitude"]]
        # condit = soup.find('p', class_="attrgroup")
        # cond = ''
        # if (condit and condit.b.text): cond = condit.b.text
    
    # stuff.loc[x] = [title, age, coord]
    stuff2["title"].append(title)
    stuff2["age"].append(age)
    stuff2["coord"].append(coord)

    time.sleep(2)
    browser.click_link_by_partial_text('next')
    
print(stuff2)

{'title': ['Foam roller', 'Sofa sleeper', 'Desk + hutch', 'Desk + hutch', 'Palm Trees', 'Dishwahser', 'Boxes', 'Aloe verde 1 sq yard  FREE', 'Oak Wood / Firewood / Free', 'FREE pine chest', 'Slider', 'FREE LAZY BOY SOFA - 2 RECLINERS - LOVE SEAT SIZE', 'FREE RV TRAVEL TRAILER', 'Horse manure compost - green', 'FREE scrap metal grill/fryer, other items', 'Free freshwater RV holding tank', "Children's room fan", 'Free freshwater RV holding tank', 'Thornless Cactus', 'Desk for free tonight and tomorrow', 'Free dining room old table and chairs', 'Free cal king bed and matches', 'Fireplace', 'FREE BOATS', 'Fire pit', 'Free night stand', 'Antique armoire', 'Free recliner sofa', 'Curb alert free', 'FREE MATTRESSES / box springs - used', 'Tree Watering Stake', 'free passion fruit tree & chili tree', 'Free Reclining Loveseat', 'Prochem truckmount frame', '2 free seats', 'Patio table, planters, emergency lanterns', 'Closet Support Brackets', 'Kenmore washer', "Thin 3' Long Bamboo Stakes", 'Fluor

In [8]:
(pd.DataFrame.from_dict(data=stuff2, orient='index')
   .to_csv('stuff.csv', header=False))

In [4]:
import scrape_cl

things = scrape_cl.scrape()
print(things)

{'title': ['Frigidaire Dryer', 'EXERCISE BENCH', 'Free business books'], 'age': ['2019-10-25T22:43:01-0700', '2019-09-30T11:31:23-0700', '2019-10-25T22:02:24-0700'], 'coord': [['38.610558', '-121.375923'], ['38.424300', '-121.437000'], ['38.534700', '-121.445100']]}


In [29]:
# Go to melissa.com and find zip codes

zips = []

look_up_url = "https://www.melissa.com/v2/lookups/latlngzip4/"
browser.visit(look_up_url)

for coords in things['coord']:
    browser.fill('lat', coords[0])
    browser.fill('lng', coords[1])
    browser.find_by_value('Submit').first.click()
    element = browser.find_by_css("tbody")
    zip_code = element.value.split("Postal Code ")[1].split("-")[0]
    zips.append(zip_code)
    time.sleep(1)
    
print(zips)

['Address 3801 El Camino Ave\nCity Sacramento\nState CA\n', '95821-6507']
['Address 8905 Laguna Springs Way\nCity Elk Grove\nState CA\n', '95758-5312\nMAK 5244172933\nPostal Carrier Route R015']
['Address 4335 52nd St\nCity Sacramento\nState CA\n', '95820-4131\nMAK 1373124119\nPostal Carrier Route C010']
['95821', '95758', '95820']


In [32]:
inc_data_url = "https://zipatlas.com/us/ca/sacramento/zip-code-comparison/median-household-income.htm"

browser.visit(inc_data_url)

inc_data = browser.find_by_css("tbody")

inc_data.value

'  Database Download\nMedian Household Income in Sacramento, CA by Zip Code\nsearch\nSacramento, CA Report:\nMedian Household Income\n$87,622.00\n$0.00\nRelated Reports\nMedian Household Income in California by Zip Code\nMedian Household Income in California by City\nMedian Household Income in the United States by Zip Code\nMedian Household Income in the United States by City\n# Zip Code Location City Population Avg. Income/H/hold National Rank\n1. 95837 38.691607, -121.602284 Sacramento, California 254 $87,622.00 #528\n2. 95829 38.486502, -121.334051 Sacramento, California 11,380 $63,774.00 #2,297\n3. 95835 38.685069, -121.543709 Sacramento, California 834 $63,611.00 #2,331\n4. 95864 38.585826, -121.376263 Sacramento, California 23,355 $59,570.00 #3,098\n5. 95831 38.494832, -121.529447 Sacramento, California 42,821 $57,329.00 #3,521\n6. 95819 38.567594, -121.437507 Sacramento, California 15,964 $55,811.00 #3,918\n7. 95830 38.490508, -121.284171 Sacramento, California 580 $53,800.00 #4

In [53]:
data_list = inc_data.value.split("National Rank")[1].split("28 total")[0]
zips = []
incomes = []
data_list = data_list.split(" ")
for i in range(1, 28):
    zips.append(data_list[8*(i-1)+1])
    incomes.append(float(data_list[8*(i-1)+7].strip('$').strip(',')))

zip_incomes = {"zip codes":zips, "incomes":incomes}
zip_incomes
df = pd.DataFrame(zip_incomes)
df.columns



ValueError: could not convert string to float: '87,622.00'